# Лабораторна робота №3
# Криптоаналіз афінної біграмної підстановки

### Реалізувати підпрограми із необхідними математичними операціями: обчисленням оберненого елементу за модулем із використанням розширеного алгоритму Евкліда, розв’язуванням лінійних порівнянь. При розв’язуванні порівнянь потрібно коректно обробляти випадок із декількома розв’язками, повертаючи їх усі.


In [9]:
from math import gcd

# Функція  реалізації розширеного алгоритму Евкліда
def extended_euclid(a, b):
    if a == 0:
        return b, 0, 1
    gcd_val, x, y = extended_euclid(b % a, a)
    return gcd_val, y - (b // a) * x, x

# функція знаходить обернений елемент числа a за модулем m
def modulo_inverse(a, m):
    if gcd(a, m) != 1:
        return "Can't find a^(-1)"
    else:
        u = extended_euclid(a, m)[1]
        return u % m
    
# Функція для вирішення лінійних конгруентних рівнянь
def modular_equation(a, b, mod): 
    d = gcd(a, mod)
    if d == 1:
        x = (modulo_inverse(a, mod) * b) % mod
        return [x]
    else:
        if b % d != 0:
            return ["no solutions"]  
        else:
            results = []
            x = modular_equation(a // d, b // d, mod // d)[0]
            for i in range(d):
                results.append((x + (mod // d) * i) % mod)
            return results

### За допомогою програми обчислення частот біграм, яка написана в ході виконання комп’ютерного практикуму No1, знайти 5 найчастіших біграм запропонованого шифртексту (за варіантом 4).

In [ ]:
from collections import Counter

# Функція для отримання всіх біграм у тексті
def get_bigrams(text): 
    bigrams = [text[i:i+2] for i in range(len(text)-1)]
    return bigrams

# Функція для пошуку найпоширеніших біграм у тексті
def find_most_common_bigrams(text, top_n=5): 
    bigrams = get_bigrams(text)
    bigram_counts = Counter(bigrams)
    total_big = sum(bigram_counts.values())
    most_common_bigrams = bigram_counts.most_common(top_n)
    return most_common_bigrams, total_big

chipter_text = open('04.txt', 'r', encoding='utf-8').read().replace("\n", "") 
most_common_bigrams, total_big = find_most_common_bigrams(chipter_text) # list

chipher_bigrams = []
for bigram, count in most_common_bigrams:
    chipher_bigrams.append(bigram)
    freq_bigr = count/total_big
    print(f"Біграм: {bigram}, Кількість: {count}, Частота {freq_bigr:.3%}")

print(chipher_bigrams)

Біграм: еш, Кількість: 68, Частота 1.163%
Біграм: шя, Кількість: 52, Частота 0.890%
Біграм: еы, Кількість: 50, Частота 0.855%
Біграм: до, Кількість: 49, Частота 0.838%
Біграм: зо, Кількість: 48, Частота 0.821%
['еш', 'шя', 'еы', 'до', 'зо']


### Перебрати можливі варіанти співставлення частих біграм мови та частих біграм шифртексту (розглядаючи пари біграм із п’яти найчастіших). Для кожного співставлення знайти можливі кандидати на ключ (a,b) шляхом розв’язання системи:

In [10]:
rus_birgams = ["ст", "но", "то", "на", "ен"]
alphabet_str = 'абвгдежзийклмнопрстуфхцчшщьыэюя'

# Формуємо всі можливі пари біграм (російські -> зашифровані)
pairs_list_bigram = [(rus_bigr, cipher_bigr) for rus_bigr in rus_birgams for cipher_bigr in chipher_bigrams]
print("Pairs:")
print(pairs_list_bigram)

# Створення словника для перетворення літер у їх числове представлення
lettertonum = {letter: index for index, letter in enumerate(alphabet_str)}

# Перетворення кожної біграми в числове представлення
pairs_num_bigram = [
    (len(alphabet_str) * lettertonum[pair[0][0]] + lettertonum[pair[0][1]], 
     len(alphabet_str) * lettertonum[pair[1][0]] + lettertonum[pair[1][1]])
    for pair in pairs_list_bigram]
print("Pairs num:")
print(pairs_num_bigram)

alphabet_squared = len(alphabet_str) ** 2
possible_key = []

# Функція для знаходження різниці двох значень
def find_delta(first_el, second_el):
    delta = first_el - second_el
    return delta

# Пошук можливих ключів
for first in range(len(pairs_num_bigram)):
    X1, Y1 = pairs_num_bigram[first]
    for second in range(first + 1, len(pairs_num_bigram)):
        X2, Y2 = pairs_num_bigram[second]
        delta_X = find_delta(X1, X2)
        delta_Y = find_delta(Y1, Y2)
        if delta_X == 0:
            continue
        try:
            result = modulo_inverse(delta_X, alphabet_squared)
            # Якщо обернений елемент знайдено
            if isinstance(result, int):
                num_delta_X = result
                a = (delta_Y * num_delta_X) % alphabet_squared
                b = (Y1 - a * X1) % alphabet_squared
                answer = modular_equation(a, b, alphabet_squared)
                if answer is not None:
                    possible_key.append((a, b))
        except ValueError as e:
            print("ValueError:", e)
            continue

print('Maybe key:')

chunk_size = 10
for i in range(0, len(possible_key), chunk_size):
    chunk = possible_key[i:i + chunk_size]
    print(chunk)




Pairs:
[('ст', 'еш'), ('ст', 'шя'), ('ст', 'еы'), ('ст', 'до'), ('ст', 'зо'), ('но', 'еш'), ('но', 'шя'), ('но', 'еы'), ('но', 'до'), ('но', 'зо'), ('то', 'еш'), ('то', 'шя'), ('то', 'еы'), ('то', 'до'), ('то', 'зо'), ('на', 'еш'), ('на', 'шя'), ('на', 'еы'), ('на', 'до'), ('на', 'зо'), ('ен', 'еш'), ('ен', 'шя'), ('ен', 'еы'), ('ен', 'до'), ('ен', 'зо')]
Pairs num:
[(545, 179), (545, 774), (545, 182), (545, 138), (545, 231), (417, 179), (417, 774), (417, 182), (417, 138), (417, 231), (572, 179), (572, 774), (572, 182), (572, 138), (572, 231), (403, 179), (403, 774), (403, 182), (403, 138), (403, 231), (168, 179), (168, 774), (168, 182), (168, 138), (168, 231)]
Maybe key:
[(0, 179), (138, 888), (503, 890), (173, 72), (390, 10), (0, 179), (761, 586), (427, 26), (571, 348), (354, 410)]
[(0, 179), (165, 588), (873, 89), (562, 448), (717, 541), (0, 179), (470, 616), (390, 10), (436, 887), (33, 453)]
[(823, 65), (0, 774), (365, 776), (35, 919), (252, 857), (200, 367), (0, 774), (627, 214), 

In [ ]:
# Створення словників для перетворення символів у числові індекси і навпаки
char_to_index = {char: i for i, char in enumerate(alphabet_str)}
index_to_char = {i: char for i, char in enumerate(alphabet_str)}

encrypted_text = chipter_text.replace('\r', '')

# Перетворюємо зашифрований текст у числовий список, замінюючи кожен символ його індексом
encrypted_numbers = [char_to_index[char] for char in encrypted_text]

# Функція для дешифрування афінного шифру
def decrypt_affine_cipher(numbers, key_a, key_b, modulus_squared):
    decrypted_message = []
    try:
        inverse_a = modulo_inverse(key_a, modulus_squared)
    except ValueError:
        print(f"Оберненого не існує для key_a = {key_a}. Пропускаємо ключ.")
        return None
    for index in range(0, len(numbers), 2):
        first_num = numbers[index]
        second_num = numbers[index + 1] if index < len(numbers) - 1 else 0 
        bigram_index = first_num * 31 + second_num 
        if not all(isinstance(value, int) for value in [inverse_a, bigram_index, key_b, modulus_squared]):
            return None
        original_index = (inverse_a * (bigram_index - key_b)) % modulus_squared
        first_original = original_index // 31
        second_original = original_index % 31
        decrypted_message.append(index_to_char[first_original % len(alphabet_str)])
        decrypted_message.append(index_to_char[second_original % len(alphabet_str)])
    return ''.join(decrypted_message)

decrypted_variants = {}
for key_a, key_b in possible_key:
    decrypted_result = decrypt_affine_cipher(encrypted_numbers, key_a, key_b, alphabet_squared)
    if decrypted_result:
        decrypted_variants[(key_a, key_b)] = decrypted_result

In [11]:
# Список рідкісних біграм
uncommon_bigrams = ["щт", "ьо", "ыж", "юв", "яы", "аы", "бй", "гй", "дй", "еы", "шщ", "шя", "щб", "щд", "щж", "ьы", "ыа", "ыь", "ыы", "ыэ"]

# Функція для розбиття тексту на біграми
def generate_bigrams(text):
    return [text[index:index + 2] for index in range(0, len(text), 2)]

# Функція для підрахунку рідкісних біграм у тексті
def calculate_uncommon_bigrams(text, uncommon_list):
    bigrams_in_text = generate_bigrams(text)
    uncommon_count = sum(1 for bigram in bigrams_in_text if bigram in uncommon_list)
    return uncommon_count

uncommon_bigram_counts = {}

for decryption_key, decoded_text in decrypted_variants.items():
    uncommon_count = calculate_uncommon_bigrams(decoded_text, uncommon_bigrams)
    uncommon_bigram_counts[decryption_key] = uncommon_count


sorted_uncommon_counts = sorted(uncommon_bigram_counts.items(), key=lambda item: item[1])
print("Топ-10 ключів із найменшою кількістю рідкісних біграм:")
for rank, (key, count) in enumerate(sorted_uncommon_counts[:10], start=1):
    print(f"{rank}. Ключ {key} - Кількість рідкісних біграм: {count}")


Топ-10 ключів із найменшою кількістю рідкісних біграм:
1. Ключ (390, 10) - Кількість рідкісних біграм: 5
2. Ключ (554, 598) - Кількість рідкісних біграм: 5
3. Ключ (862, 108) - Кількість рідкісних біграм: 7
4. Ключ (503, 890) - Кількість рідкісних біграм: 12
5. Ключ (641, 960) - Кількість рідкісних біграм: 14
6. Ключ (376, 489) - Кількість рідкісних біграм: 14
7. Ключ (221, 489) - Кількість рідкісних біграм: 16
8. Ключ (80, 788) - Кількість рідкісних біграм: 17
9. Ключ (771, 536) - Кількість рідкісних біграм: 18
10. Ключ (571, 351) - Кількість рідкісних біграм: 18


In [ ]:
final_key = (390, 10)

# Отримуємо розшифрований текст за фінальним ключем
decrypted_text = decrypted_variants[final_key]
output_file_name = "decrypted_text.txt"
with open(output_file_name, "w", encoding="utf-8") as output_file:
    output_file.write(decrypted_text)

print(decrypted_text)

print(f"Розшифрований текст збережено у файл: {output_file_name}")

еслиправдачтодостоевскийвсибиринебылподверженприпадкамтоэтолишьподтверждаетточтоегоприпадкибылиегокаройонболеевнихненуждалсякогдабылкараеминымобразомнодоказатьэтоневозможноскорееэтойнеобходимостьювнаказаниидляпсихическойэкономиидостоевскогообясняетсяточтоонпрошелнесломленнымчерезэтигодыбедствийиуниженийосуждениедостоевскоговкачествеполитическогопреступникабылонесправедливымиондолженбылэтознатьноонпринялэтонезаслуженноенаказаниеотбатюшкицарякакзаменунаказаниязаслуженногоимзасвойгрехпоотношениюксвоемусобственномуотцувместосамонаказанияондалсебянаказатьзаместителюотцаэтодаетнамнекотороепредставлениеопсихологическомоправданиинаказанийприсуждаемыхобществомэтонасамомделетакмногиеизпреступниковжаждутнаказанияеготребуетихсверхяизбавляясебятакимобразомотсамонаказаниятотктознаетсложноеиизменчивоезначениеистерическихсимптомовпойметчтомыздесьнепытаемсядобитьсясмыслаприпадковдостоевскогововсейполнотедостаточнотогочтоможнопредположитьчтоихпервоначальнаясущностьосталасьнеизменнойнесморянавсепоследующ